In [1]:
# import the relevant libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics

In [2]:
data = pd.read_csv('cleaned_car_data.csv')
data.head()

,name,manufacturer,year,age,mileage,engine,transmission,price
0,Mazda MX5,Mazda,2007,14,63131,Petrol,Manual,7499
1,Jaguar XF,Jaguar,2010,11,61890,Petrol,Automatic,7775
2,Audi A6,Audi,2012,9,129170,Diesel,Automatic,6950
3,Nissan Qashqai,Nissan,2013,8,44900,Petrol,Automatic,7790
4,MINI Mini,Mini,2017,4,32012,Petrol,Manual,15999


In [3]:
#drop the name and year columns because it is irrelevant in our model building
data = data.drop(['name', 'year'], axis=1)

In [4]:
data.head()

,manufacturer,age,mileage,engine,transmission,price
0,Mazda,14,63131,Petrol,Manual,7499
1,Jaguar,11,61890,Petrol,Automatic,7775
2,Audi,9,129170,Diesel,Automatic,6950
3,Nissan,8,44900,Petrol,Automatic,7790
4,Mini,4,32012,Petrol,Manual,15999


In [5]:
# get dummny data
data = pd.get_dummies(data)
data

,age,mileage,price,manufacturer_Abarth,manufacturer_Alfa-Romero,manufacturer_Audi,manufacturer_BMW,manufacturer_Bentley,manufacturer_Chevrolet,manufacturer_Chrysler,...,manufacturer_Volkswagen,manufacturer_Volvo,engine_Diesel,engine_Electric,engine_Hybrid,engine_Petrol,engine_Plug_in_hybrid,transmission_Automatic,transmission_Manual,transmission_Semiautomatic
0,14,63131,7499,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,11,61890,7775,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,9,129170,6950,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,8,44900,7790,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,4,32012,15999,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,1,10290,22000,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2982,1,16193,27000,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2983,4,59926,16000,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2984,1,12355,30000,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [6]:
# creating X and y variables
X = data.drop('price', axis=1)
y = np.log(data['price'])

In [7]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2090, 50), (896, 50), (2090,), (896,))

In [9]:
# feature scale the X_train and X_test values

scale = StandardScaler().fit(X_train)

# transform training data
X_train = scale.transform(X_train)

# transform testing data
X_test = scale.transform(X_test)

print(X_train)
print('\n')
print(X_test)

[[-0.21452177 -0.47546652 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]
 [-1.16913606 -1.09127695 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]
 [-1.48734082 -1.05419589 -0.05365705 ... -0.55266093 -1.3256153
   2.6018295 ]
 ...
 [ 1.69470679  0.37176945 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]
 [ 0.10368299  0.04714465 -0.05365705 ... -0.55266093  0.75436667
  -0.38434494]
 [ 1.37650203 -0.28906798 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]]


[[-1.16913606 -1.09339587 -0.05365705 ... -0.55266093  0.75436667
  -0.38434494]
 [-0.53272654 -0.84574737 -0.05365705 ... -0.55266093 -1.3256153
   2.6018295 ]
 [-1.16913606 -1.10356005 -0.05365705 ... -0.55266093 -1.3256153
   2.6018295 ]
 ...
 [ 2.96752584  3.26574741 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]
 [ 1.05829727  0.45155994 -0.05365705 ...  1.80942771 -1.3256153
  -0.38434494]
 [-1.16913606 -1.03969455 -0.05365705 ... -0.55266093  0.75436667
  -0.38434494]]


In [20]:
#linear regression
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [11]:
#Ridge regression
rg = linear_model.Ridge(alpha=.5)
rg.fit(X_train, y_train)

Ridge(alpha=0.5)

In [12]:
#Random Forest Regressor
rf=  RandomForestRegressor(random_state=123)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=123)

In [13]:
#AdaBoost
ab =  AdaBoostRegressor(random_state=123)
ab.fit(X_train, y_train)

AdaBoostRegressor(random_state=123)

In [14]:
#Suport Vector Regressor
sv =  SVR()
sv.fit(X_train, y_train)

SVR()

In [15]:
#lasso regression
ls = linear_model.Lasso(alpha=0.1)
ls.fit(X_train, y_train)

Lasso(alpha=0.1)

In [16]:
#Bayesian Regression
br = linear_model.BayesianRidge()
br.fit(X_train, y_train)

BayesianRidge()

In [17]:
train_score = pd.DataFrame({
    'model': ['Linear Regression', 'Ridge Regression', 'Random Forest', 'AdaBoost Regressor', 'Support Vector Regression',
              'Lasso regression','Bayesian Regression'],
    
    'score': [lr.score(X_train, y_train), rg.score(X_train, y_train),
                rf.score(X_train, y_train) , ab.score(X_train, y_train),
                sv.score(X_train, y_train), ls.score(X_train, y_train),
                br.score(X_train, y_train)]
    })
train_score            

,model,score
0,Linear Regression,0.784242
1,Ridge Regression,0.784251
2,Random Forest,0.964993
3,AdaBoost Regressor,0.688028
4,Support Vector Regression,0.826387
5,Lasso regression,0.556646
6,Bayesian Regression,0.784204


In [18]:
lr_pred = lr.predict(X_test)
rg_pred = rg.predict(X_test)
rf_pred = rf.predict(X_test)
ab_pred = ab.predict(X_test)
sv_pred = sv.predict(X_test)
ls_pred = ls.predict(X_test)
br_pred = br.predict(X_test)

In [19]:
# Generalisation
best_model = pd.DataFrame({
    'model': ['Linear Regression', 'Ridge Regression', 'Random Forest', 'AdaBoost Regressor', 'Support Vector Regression',
              'Lasso regression','Bayesian Regression'],
    
    'mae': [mean_absolute_error(y_test, lr_pred), mean_absolute_error(y_test, rg_pred),
           mean_absolute_error(y_test, rf_pred), mean_absolute_error(y_test, ab_pred),
           mean_absolute_error(y_test, sv_pred), mean_absolute_error(y_test, ls_pred),
           mean_absolute_error(y_test, br_pred)],
    
    'mse': [mean_squared_error(y_test, lr_pred), mean_squared_error(y_test, rg_pred),
            mean_squared_error(y_test, rf_pred), mean_squared_error(y_test, ab_pred),
            mean_squared_error(y_test, sv_pred), mean_squared_error(y_test, ls_pred),
            mean_squared_error(y_test, br_pred)],
    
    'rmse': [(np.sqrt(metrics.mean_squared_error(y_test, lr_pred))),(np.sqrt(metrics.mean_squared_error(y_test, rg_pred))),
             (np.sqrt(metrics.mean_squared_error(y_test, rf_pred))), (np.sqrt(metrics.mean_squared_error(y_test, ab_pred))),
             (np.sqrt(metrics.mean_squared_error(y_test, sv_pred))), (np.sqrt(metrics.mean_squared_error(y_test, ls_pred))),
             (np.sqrt(metrics.mean_squared_error(y_test, br_pred)))]
})
best_model

,model,mae,mse,rmse
0,Linear Regression,1.076619e+09,1.038561e+21,3.222671e+10
1,Ridge Regression,2.118539e-01,1.046032e-01,3.234241e-01
2,Random Forest,2.119042e-01,7.792599e-02,2.791523e-01
3,AdaBoost Regressor,2.618770e-01,1.023349e-01,3.198983e-01
4,Support Vector Regression,1.961090e-01,6.910451e-02,2.628774e-01
5,Lasso regression,3.058535e-01,1.578246e-01,3.972715e-01
6,Bayesian Regression,2.120740e-01,1.037862e-01,3.221587e-01


In [22]:
# Tune the model

rf = RandomForestRegressor(random_state=123)
rf_params = dict(
    n_estimators=np.arange(80, 162, 10), max_depth=[10, 12, 14, 16, 18, None])
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state = 0)

gs_rf = GridSearchCV(rf, rf_params, n_jobs=-1, scoring="neg_root_mean_squared_error", cv = cv)

# fit the model
gs_rf.fit(X_train, y_train)

gs_rf.best_params_

{'max_depth': 12, 'n_estimators': 160}

In [23]:
rf = RandomForestRegressor(max_depth=10, n_estimators=160, random_state=123)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [24]:
model = pd.DataFrame({
    'model': ['Random Forest'],
    'mae': [mean_absolute_error(y_test, rf_pred)],
    'mse': [mean_squared_error(y_test, rf_pred)],
    'rmse': [np.sqrt(metrics.mean_squared_error(y_test, rf_pred))]
    })
model

,model,mae,mse,rmse
0,Random Forest,0.210495,0.075795,0.275309


In [42]:
new_data = [['Volksagen', 7, 60000, 'Petrol', 'Manual']]
# convert to array
new_data = np.array(new_data)
new_data

array([['Volksagen', '7', '60000', 'Petrol', 'Manual']], dtype='<U9')

In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
le = LabelEncoder()

In [45]:
new_data[:,0] = le.fit_transform(new_data[:,0])
new_data[:,3] = le.fit_transform(new_data[:,3])
new_data[:,4] = le.fit_transform(new_data[:,4])
new_data

array([['0', '7', '60000', '0', '0']], dtype='<U9')

In [46]:
new_data = scale.fit_transform(new_data)

In [47]:
new_data

array([[0., 0., 0., 0., 0.]])

In [37]:
price = rf.predict(new_data)
act_price = np.exp(price) + 1
act_price = round(act_price[0])  # round to the neaarest Naira

print(f"The estimated cost of the car is GBP {act_price:,}")

ValueError: Number of features of the model must match the input. Model n_features is 50 and input n_features is 5 